# ChatGPT API in ython
This notebook aims to provide a simple example of interacting with ChatGPT  using its python API

In [1]:
# Install Open AI API package
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 10.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [ ]:
ls /drive/MyDrive/'Colab Notebooks'

In [3]:
import openai
import os.path
import pandas as pd

if os.path.isfile("/drive/MyDrive/Colab Notebooks/openai.api_key"):
    openai.api_key = pd.read_csv("/drive/MyDrive/Colab Notebooks/openai.api_key", header=None).loc[0,0]
    print("Your open AI API key was obtained from /drive/MyDrive/'Colab Notebooks'/openai.api_key")
else:
    from getpass import getpass
    secret = getpass('Enter the openai passcode value (can be obtained from https://platform.openai.com/account/api-keys): ')
    openai.api_key = secret
    !echo {secret} > openai.api_key
    !cp openai.api_key /drive/MyDrive/'Colab Notebooks'/openai.api_key
    print("Your open AI API key was copied to /drive/MyDrive/Colab Notebooks/openai.api_key")

Enter the openai passcode value (can be obtained from https://platform.openai.com/account/api-keys): ··········
Your open AI API key was copied to /drive/MyDrive/Colab Notebooks/openai.api_key


In [ ]:
messages=[{"role": "user", "content": "Turn this movie title into emojis: Harry Potter"}]

responses = openai.ChatCompletion.create(
    model="gpt-4",
    max_tokens=50,
    temperature=.1,  # What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
    messages = messages
    )

response = responses.choices[0].message.content
print(response)

⚡🧙‍♂️📚


## Bringing a list of highly rated movies from MovieLens
Data source: [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/)

25 million ratings - 62,000 movies by 162,000 users - Released 12/2019

In [ ]:
# Download movie 25M lens dataset
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2023-03-31 18:29:57--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  80.9MB/s    in 3.3s    

2023-03-31 18:30:01 (76.7 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [ ]:
# Unzip ml-25m.zip
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
ls -lh ml-25m/

total 1.1G
-rw-rw-r-- 1 root root 416M Nov 21  2019 genome-scores.csv
-rw-rw-r-- 1 root root  18K Nov 21  2019 genome-tags.csv
-rw-rw-r-- 1 root root 1.4M Nov 21  2019 links.csv
-rw-rw-r-- 1 root root 2.9M Nov 21  2019 movies.csv
-rw-rw-r-- 1 root root 647M Nov 21  2019 ratings.csv
-rw-rw-r-- 1 root root  11K Nov 21  2019 README.txt
-rw-rw-r-- 1 root root  38M Nov 21  2019 tags.csv


In [ ]:
movies = pd.read_csv('ml-25m/movies.csv', encoding='latin-1') # encoding='latin-1' was used
ratings = pd.read_csv('ml-25m/ratings.csv') # tab separated data

ratings = ratings.groupby('movieId').agg({'rating': 'mean', 'userId': 'count'})
ratings.columns = ['avg_rating', 'n_rating']
ratings = movies[['movieId', 'title']].merge(ratings[ratings.n_rating >= 10000].reset_index(), on='movieId').sort_values('avg_rating', ascending=False)
ratings = ratings.drop('movieId', axis=1).reset_index(drop=True)
ratings.head(10)

,title,avg_rating,n_rating
0,"Shawshank Redemption, The (1994)",4.413576,81482
1,"Godfather, The (1972)",4.324336,52498
2,"Usual Suspects, The (1995)",4.284353,55366
3,"Godfather: Part II, The (1974)",4.261759,34188
4,Seven Samurai (Shichinin no samurai) (1954),4.254769,13367
5,Schindler's List (1993),4.247579,60411
6,12 Angry Men (1957),4.243014,16569
7,Rear Window (1954),4.237948,20162
8,Fight Club (1999),4.228311,58773
9,One Flew Over the Cuckoo's Nest (1975),4.218662,36058


In [ ]:
def GPTCompletion(messages, max_tokens):
    responses = openai.ChatCompletion.create(
        model="gpt-4",
        max_tokens=max_tokens,
        temperature=.5,  # What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
        messages = messages
        )
    response = responses.choices[0].message.content

    return response

In [ ]:
pd.set_option('display.max_colwidth', None)  # to show the full text in summary
enoji_df = pd.DataFrame()

for movie in ratings.title[:10]:
    messages=[{"role": "user", "content": "Turn this movie title into emojis using the movie script and not just the title: {}".format(movie)}]
    emoji_response = GPTCompletion(messages, max_tokens=20)
    
    messages=[{"role": "user", "content": "Provide a one line summary about the the movie title: {}. Max 20 words".format(movie)}]
    summary_response = GPTCompletion(messages, max_tokens=100)
    
    enoji_df = pd.concat([enoji_df, pd.DataFrame({'movie': [movie], 'emoji': [emoji_response], 'summary': [summary_response]})], ignore_index=True)
    
enoji_df

,movie,emoji,summary
0,"Shawshank Redemption, The (1994)",🏢🔒🚶‍♂️👮‍,"Wrongfully convicted man forms friendships and finds hope in prison, ultimately escaping to redemption."
1,"Godfather, The (1972)",🇮🇹🕴️👨‍👧‍,"The Godfather is a 1972 crime drama about a powerful Italian-American Mafia family, led by patriarch Vito Corleone."
2,"Usual Suspects, The (1995)",👥🔍🚢💥🚬💼💰,"A twisting crime thriller involving a heist, a mysterious mastermind, and a lineup of five seemingly unconnected criminals."
3,"Godfather: Part II, The (1974)",🇮🇹🕴️👨‍👦,The Godfather: Part II (1974) chronicles Michael Corleone's rise to power and his father's early mafia days.
4,Seven Samurai (Shichinin no samurai) (1954),🎬: 7️⃣🤺 (🇯🇵,"Epic Japanese film about seven samurai defending a village from bandits, showcasing themes of loyalty, sacrifice, and honor."
5,Schindler's List (1993),🇩🇪📃👨🏻‍💼,"""Schindler's List"" (1993) is a powerful, Oscar-winning film about a German businessman who saves over a thousand Polish Jews during the Holocaust."
6,12 Angry Men (1957),1️⃣2️⃣😠🧑‍⚖,"12 Angry Men (1957): Jurors deliberate a seemingly open-and-shut case, revealing personal biases and questioning justice."
7,Rear Window (1954),🔙🪟🧑‍🦽📷🔎,Rear Window is a 1954 suspense thriller about a wheelchair-bound photographer who becomes convinced his neighbor has committed murder.
8,Fight Club (1999),👊🤼‍♂️🤝🔥💣,An insomniac office worker forms an underground fight club to combat his existential crisis and societal disillusionment.
9,One Flew Over the Cuckoo's Nest (1975),🏥🤪1️⃣✈️🕊️,"A rebellious patient challenges oppressive authority in a mental institution, inspiring fellow patients to embrace life."
